# Airnow Highest 5 Notifications

Runs every five minutes, and sends email notifications to a small set of people whenever a particular set of cities appears in, changes position in, or leaves the Highest 5.  Users and cites of concern are defined in JSON file `highest-5-notifications-config.json`.  Format for that configuration file is like this, where city IDs are the Airnow City IDs found in `airnow_city_id_to_city_info.json`:

```
{
   "alertable_cities" : {
      "162" : {
         "emails" : ["foo@bar.com"]
      },
      "164" : {
         "emails" : ["a@z.com", "foo@bar.com"]
      }
   }
}
```

This script uses MailGun to send emails and depends on a configuration file named `mailgun-config.json` in the current directory.  Format for that file is:

```
{
   "api_key" : "YOUR_API_KEY_HERE",
   "domain_name" : "YOUR_DOMAIN_NAME_HERE"
}
```

Reports to stat.createlab.org as `Airnow Highest Five - Notifications`.

Airnow's docs for the highest 5 are here: https://airnow.gov/index.cfm?action=airnow.news_item&newsitemid=103

In [0]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat
from dateutil import rrule, tz, parser

In [0]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())

exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [0]:
STAT_SERVICE_NAME = 'Airnow Highest Five - Notifications'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-highest-five-notifications'

RUN_INTERVAL_SECONDS = 60 * 5   # every 5 minutes

CONFIG_FILE = './highest-5-notifications-config.json'

CITY_INFO_JSON_FILE = AirnowCommon.HIGHEST_FIVE_AQI_DIRECTORY + '/airnow_city_id_to_city_info.json'

MAILGUN_CONFIG_FILE = './mailgun-config.json'
MAILGUN_CONFIG = {}
with open(MAILGUN_CONFIG_FILE, 'r') as f:
    MAILGUN_CONFIG = json.load(f)

In [0]:
Stat.set_service(STAT_SERVICE_NAME)

In [0]:
city_info = {}

notifications_config = {}

previous_rankings_str = ""
previous_alertable_city_rankings = {}

current_rankings = []

In [0]:
def read_city_info():
    global city_info
    with open(CITY_INFO_JSON_FILE, 'r') as f:
        city_info = json.load(f)

# read_city_info()
# print(json.dumps(city_info, sort_keys=True, indent=3))

In [0]:
def get_city_and_state(city_id):
    global city_info
    city_id = str(city_id)
    if city_id in city_info:
        info = city_info[city_id]
        return info['city'] + ', ' + info['state']

    return "Unknown City"

# get_city_and_state(162)
# get_city_and_state("162")

In [0]:
def read_config_file():
    global notifications_config
    with open(CONFIG_FILE, 'r') as f:
        notifications_config = json.load(f)

#read_config_file()
#print(json.dumps(notifications_config, sort_keys=True, indent=3))

In [0]:
def is_alertable_city(city_id):
    global notifications_config
    return str(city_id) in notifications_config['alertable_cities']

# print(is_alertable_city(None))
# print(is_alertable_city(162))
# print(is_alertable_city("162"))

In [0]:
def get_email_addresses_to_notify_for_city(city_id):
    global notifications_config
    city_id_str = str(city_id)
    if city_id_str in notifications_config['alertable_cities']:
        return notifications_config['alertable_cities'][city_id_str]['emails']

    return []

# get_email_addresses_to_notify_for_city("162")

In [0]:
def check_current_highest_five():
    global previous_rankings_str, current_rankings
    files = glob.glob(AirnowCommon.HIGHEST_FIVE_AQI_DAT_DIRECTORY + '/[0-9]*.dat')
    if len(files) == 0:
        return None
    last_file = sorted(files)[-1]
    Stat.debug('Most recent data file is %s' % (last_file), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    # reset the current rankings
    current_rankings = []

    # read last line from file
    last_line = None
    with open(last_file, 'r') as f:
        for line in f:
            pass
        last_line = line

    # now parse the last line
    if last_line:
        Stat.debug('Most recent rankings: %s' % (last_line), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

        # Record format example: 1583863202.348545:1,235,111|2,809,93|3,789,91|4,946,86|5,230,81
        # A colon separates the Unix timestamp from the rankings.  Rankings are pipe delimited and there should exist 5 per timestamp.
        # A ranking item consists of three comma-delimited values: the rank index [1-5], the Airnow city ID, and the AQI
        try:
            (timestamp, rankings) = last_line.split(':')
            timestamp = float(timestamp)

            # Do a quickie string comparison to make sure the rankings have actually changed.  If so, then
            # we'll do a deeper check later to make sure the alertable rankings have changed, but ignoring
            # changes in only AQI.  It's just the rankings we care about.
            if rankings == previous_rankings_str:
                Stat.debug('Rankings unchanged, nothing to do', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            else:
                Stat.debug('Rankings have changed, checking for alertable cities', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
                previous_rankings_str = rankings

                # parse the rankings, building a map of city_id to rank
                alertable_city_rankings = {}
                for ranking in rankings.split('|'):
                    (rank, city_id, aqi) = map(int,ranking.split(','))

                    current_rankings.append({"rank": rank, "city_id": city_id, "city_name": get_city_and_state(city_id), "aqi": aqi})

                    # only bother remembering it if it's an alertable city
                    if is_alertable_city(city_id):
                        alertable_city_rankings[city_id] = rank

                return alertable_city_rankings

        except:
            Stat.warning('Failed to parse most recent rankings. Skipping.', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    return None

In [0]:
def build_alerts(alertable_city_rankings):
    global previous_alertable_city_rankings
    alerts_by_city = {}

    if alertable_cities_in_highest_five != None:
        # start by checking whether any cities which were previously in the highest 5 have dropped out
        for city_id in previous_alertable_city_rankings:
            if city_id not in alertable_city_rankings:
                # create a notification that this city has dropped out of the highest 5
                city_name = get_city_and_state(city_id)
                alerts_by_city[city_id] = {'brief' : "%s: --> out" % (city_name),
                                           'verbose' : "%s is no longer in the Highest Five" % (city_name)}

        # now check cities in the current highest 5 to see whether they're new or they have changed rank
        for city_id in alertable_city_rankings:
            city_name = get_city_and_state(city_id)
            current_rank = alertable_city_rankings[city_id]
            if city_id in previous_alertable_city_rankings:
                # see whether the rank has changed
                previous_rank = previous_alertable_city_rankings[city_id]
                if previous_rank != current_rank:
                    # create an alert that the rank has changed
                    alerts_by_city[city_id] = {'brief' : "%s: %d --> %d" % (city_name, previous_rank, current_rank),
                                               'verbose' : "%s has changed from rank %d to rank %d" % (city_name, previous_rank, current_rank)}
            else:
                # create a notification that this city is now in the highest 5
                alerts_by_city[city_id] = {'brief' : "%s: --> %d" % (city_name, current_rank),
                                           'verbose' : "%s has entered at rank %d" % (city_name, current_rank)}

        previous_alertable_city_rankings = alertable_city_rankings

    return alerts_by_city

In [0]:
def send_email(to_address, alerts):
    global current_rankings
    if len(alerts) > 0:
        if len(alerts) == 1:
            city_or_cities = "A city"
            has_or_have = "has"
        else:
            city_or_cities = "Cities"
            has_or_have = "have"

        email_body = city_or_cities + " you're watching "+has_or_have+" new activity in the Airnow Highest Five:\n\n"
        for alert in alerts:
            email_body += "   %s\n" % (alert['verbose'])
        email_body += "\nThe current rankings are:\n\n"
        for ranking in current_rankings:
            email_body += "   %d: %s   (AQI %d)\n" % (ranking['rank'], ranking['city_name'], ranking['aqi'])
        email_body += "\nView details at https://airstats.createlab.org/highest-five/"

        return requests.post(
            "https://api.mailgun.net/v3/"+MAILGUN_CONFIG['domain_name']+"/messages",
            auth=("api", MAILGUN_CONFIG['api_key']),
            data={"from": "Airnow Highest Five Alerts <mailgun@"+MAILGUN_CONFIG['domain_name']+">",
                  "to": [to_address],
                  "subject": '[AHFA] ' + ' | '.join(list(map(lambda alert: alert['brief'], alerts))),
                  "text": email_body})

# send_email("bartley@cmu.edu", [{'brief': 'Hidden Valley, AZ: --> 1', 'verbose': 'Hidden Valley, AZ has entered at rank 1'}])
# send_email("bartley@cmu.edu", [{'brief': 'Hidden Valley, AZ: --> 1', 'verbose': 'Hidden Valley, AZ has entered at rank 1'}, {'brief': 'Mississippi Gulf Coast, MS: --> 3', 'verbose': 'Mississippi Gulf Coast, MS has entered at rank 3'}])

In [0]:
def send_alerts(alerts_by_city):
    num_cities = len(alerts_by_city)
    if (num_cities > 0):
        Stat.debug('Sending alerts for %d %s' % (num_cities, "city" if (num_cities == 1) else "cities"), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

        alerts_by_email_address = {}
        for city_id in alerts_by_city:
            alert = alerts_by_city[city_id]
            Stat.debug("City ID [%s]: %s" % (city_id, alert['brief']), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            email_addresses = get_email_addresses_to_notify_for_city(city_id)
            for email_address in email_addresses:
                if email_address not in alerts_by_email_address:
                    alerts_by_email_address[email_address] = []

                alerts_by_email_address[email_address].append(alert)

        # send the emails
        for email in alerts_by_email_address:
            print("Sending these alerts to email [%s]:" % (email))
            for alert in alerts_by_email_address[email]:
                print("   %s" % alert['brief'])
            send_email(email, alerts_by_email_address[email])

In [0]:
def run():
    global current_rankings

    Stat.info('Checking Highest 5 for changes which would trigger notifications...', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    start_time = time.time()

    # reload the city info so we're sure to pick up any changes
    read_city_info()

    # reload the config file. Do this every run to make it easy to change the set of alertable cities and/or users to notify
    read_config_file()

    # get a map of alertable cities (if any) in the current highest five
    alertable_cities_in_highest_five = check_current_highest_five()

    if alertable_cities_in_highest_five != None:
        print(json.dumps(alertable_cities_in_highest_five, sort_keys=True, indent=3))

        # build alerts
        alerts_by_city = build_alerts(alertable_cities_in_highest_five)

        # send alerts, if any
        send_alerts(alerts_by_city)
    else:
        Stat.debug('None of the cities in the highest 5 require notification', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    end_time = time.time()
    Stat.up('Done with highest five notifications!', details='Took %.1f seconds' % (end_time - start_time), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=RUN_INTERVAL_SECONDS*1.5)

def run_forever():
    while True:
        run()
        sleep_until_next_period(RUN_INTERVAL_SECONDS, 1*60)  # start at 1 minutes after the hour

run_forever()